In [1]:
import os
os.chdir('..')

# Import necessary modules

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Read dataset

In [3]:
from sklearn.preprocessing import MinMaxScaler

In [4]:
species_map = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}

In [5]:
df = pd.read_csv('example/data.csv')

In [6]:
df_X = df.drop(columns='species')
df_y = df['species'].map(species_map)

In [7]:
df_X

,sepal-length,sepal-width,petal-length,petal-width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [8]:
df_y

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Name: species, Length: 150, dtype: int64

In [9]:
x = df_X.values 
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_X = pd.DataFrame(x_scaled, columns=df_X.columns)

In [10]:
df_X

,sepal-length,sepal-width,petal-length,petal-width
0,0.222222,0.625000,0.067797,0.041667
1,0.166667,0.416667,0.067797,0.041667
2,0.111111,0.500000,0.050847,0.041667
3,0.083333,0.458333,0.084746,0.041667
4,0.194444,0.666667,0.067797,0.041667
...,...,...,...,...
145,0.666667,0.416667,0.711864,0.916667
146,0.555556,0.208333,0.677966,0.750000
147,0.611111,0.416667,0.711864,0.791667
148,0.527778,0.583333,0.745763,0.916667


# Create linguistic variables

In [11]:
from doggos.knowledge import LinguisticVariable, Domain

In [12]:
# features linguistic variables
sepal_length = LinguisticVariable('sepal-length', Domain(0, 1.001, 0.001))
sepal_width = LinguisticVariable('sepal-width', Domain(0, 1.001, 0.001))
petal_length = LinguisticVariable('petal-length', Domain(0, 1.001, 0.001))
petal_width = LinguisticVariable('petal-width', Domain(0, 1.001, 0.001))

In [13]:
# decision linguistic variable
species = [LinguisticVariable('0', Domain(0, 1, 0.001)), 
           LinguisticVariable('1', Domain(0, 1, 0.001)), 
           LinguisticVariable('2', Domain(0, 1, 0.001))]

# Define clauses

In [14]:
from doggos.knowledge import Clause
from doggos.fuzzy_sets import IntervalType2FuzzySet
from doggos.utils.membership_functions import triangular, trapezoidal, gaussian

In [15]:
clauses = [
    # clauses for petal length
    Clause(petal_length, 'small', IntervalType2FuzzySet(triangular(0, 0.25, 0.5, 0.8), triangular(0, 0.25, 0.5))),
    Clause(petal_length, 'medium', IntervalType2FuzzySet(triangular(0.25, 0.5, 0.75, 0.8), triangular(0.25, 0.5, 0.75))),
    Clause(petal_length, 'big', IntervalType2FuzzySet(triangular(0.5, 0.75, 1, 0.8), triangular(0.5, 0.75, 1, 1))),
    # clauses for petal width
    Clause(petal_width, 'small', IntervalType2FuzzySet(gaussian(0.25, 0.1), gaussian(0.25, 0.15))),
    Clause(petal_width, 'medium', IntervalType2FuzzySet(gaussian(0.5, 0.1), gaussian(0.5, 0.15))),
    Clause(petal_width, 'big', IntervalType2FuzzySet(gaussian(0.75, 0.1), gaussian(0.75, 0.15))),
    # claueses for sepal length
    Clause(sepal_length, 'small', IntervalType2FuzzySet(triangular(0, 0.25, 0.5, 0.8), triangular(0, 0.25, 0.5))),
    Clause(sepal_length, 'medium', IntervalType2FuzzySet(triangular(0.25, 0.5, 0.75, 0.8), triangular(0.25, 0.5, 0.75))),
    Clause(sepal_length, 'big', IntervalType2FuzzySet(triangular(0.6, 0.9, 1, 0.8), triangular(0.6, 0.9, 1, 1))),
    # clauses for sepal width
    Clause(sepal_width, 'small', IntervalType2FuzzySet(gaussian(0.25, 0.1), gaussian(0.25, 0.15))),
    Clause(sepal_width, 'medium', IntervalType2FuzzySet(gaussian(0.5, 0.1), gaussian(0.5, 0.15))),
    Clause(sepal_width, 'big', IntervalType2FuzzySet(gaussian(0.75, 0.1), gaussian(0.75, 0.15)))
]

In [16]:
for clause in clauses:
    print(clause)
    print('asd')

Clause petal-length is small
asd
Clause petal-length is medium
asd
Clause petal-length is big
asd
Clause petal-width is small
asd
Clause petal-width is medium
asd
Clause petal-width is big
asd
Clause sepal-length is small
asd
Clause sepal-length is medium
asd
Clause sepal-length is big
asd
Clause sepal-width is small
asd
Clause sepal-width is medium
asd
Clause sepal-width is big
asd


# Define consequents

In [17]:
from doggos.knowledge.consequents import TakagiSugenoConsequent

In [18]:
parameters_1 = {sepal_length: 0.25, sepal_width: 0.25, petal_length: 0.5, petal_width: 0.5}
parameters_2 = {sepal_length: 0.15, sepal_width: 0.35, petal_length: 0.15, petal_width: 0.35}
parameters_3 = {sepal_length: 0.35, sepal_width: 0.15, petal_length: 0.35, petal_width: 0.15}
consequent_1 = TakagiSugenoConsequent(parameters_1, 0, species[0])
consequent_2 = TakagiSugenoConsequent(parameters_2, 0, species[0])
consequent_3 = TakagiSugenoConsequent(parameters_3, 0, species[0])

# Define terms

In [19]:
from doggos.knowledge import Term
from doggos.algebras import LukasiewiczAlgebra

In [20]:
terms = list()
algebra = LukasiewiczAlgebra()
for clause in clauses:
    terms.append(Term(algebra, clause))

# Define antecedents

In [1]:
# define random antecedents
antecedent1 = eval('terms[0] & terms[3] | terms[5]')
antecedent2 = (terms[1] & terms[4]) | (terms[3] & terms[8])
antecedent3 = terms[2] & (terms[0] & terms[7] & (terms[3] | terms[6]) 
antecedent3+= | terms[8])

SyntaxError: invalid syntax (<ipython-input-1-bcaee32ecb8d>, line 5)

# Create rules

In [22]:
from doggos.knowledge import Rule

In [23]:
rules = [
    Rule(antecedent1, consequent_1),
    Rule(antecedent2, consequent_2),
    Rule(antecedent3, consequent_3)
]

# Fuzzify dataset

In [24]:
from doggos.knowledge import fuzzify

In [25]:
df_X_fuzzified = fuzzify(df_X, clauses)

In [26]:
data = df_X.reset_index().to_dict(orient='list')
data.pop('index', None)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149]

In [28]:
measures = {sepal_length: data['sepal-length'], 
         sepal_width: data['sepal-width'], 
         petal_length: data['petal-length'], 
         petal_width: data['petal-width']}

# Create inference system

In [29]:
from doggos.inference import TakagiSugenoInferenceSystem
from doggos.inference.defuzzification_algorithms import takagi_sugeno_karnik_mendel

In [30]:
inference_system = TakagiSugenoInferenceSystem(rules)

In [31]:
result = inference_system.infer(takagi_sugeno_karnik_mendel, df_X_fuzzified, measures)

In [32]:
result

{<doggos.knowledge.linguistic_variable.LinguisticVariable at 0x230d776feb0>: [0.24000614955634159,
  0.1877356580523235,
  0.17502477416560183,
  0.1700002526332189,
  0.24004402745729933,
  0.3780583358358337,
  0.20501261030818127,
  0.23000303194712973,
  0.12998745408206044,
  0.17500000000000002,
  0.2850904851531022,
  0.2150355271305655,
  0.15500350612753397,
  0.10508119188633203,
  0.3300653830344781,
  0.4000003743782887,
  0.32004020107440767,
  0.250026343906766,
  0.36500073916895837,
  0.2850137500867378,
  0.2877207241261426,
  0.2850001502531282,
  0.19964969433773505,
  0.3225725681536129,
  0.2526099855231708,
  0.20774653683068112,
  0.2899380180056814,
  0.2500161929162813,
  0.23500776491908854,
  0.19000000000000003,
  0.20519208776822379,
  0.3145452277147051,
  0.30029398076406066,
  0.34001638965264325,
  0.17500000000000002,
  0.19500182085628187,
  0.26500081209054466,
  0.17500000000000002,
  0.13500772885772072,
  0.23500452998633797,
  0.24000650767317422

# Make classification

In [33]:
from sklearn.metrics import accuracy_score

In [109]:
theta1 = 0.38
theta2 = 0.72

In [110]:
def classify(theta1, theta2):
    def _classify(x):
        if x < theta1:
            return 0
        elif theta1 <= x < theta2:
            return 1
        else:
            return 2
    return _classify

In [111]:
classify_func = classify(theta1, theta2)
y_pred = list(eval('map(lambda x: classify_func(x), result[species[0]]'))

In [112]:
accuracy = accuracy_score(y_pred, df_y.values)
print(f'Accuracy: {accuracy:.5f}')

Accuracy: 0.89333
